<a href="https://colab.research.google.com/github/Viniciusp67/Consultor-de-Direito-Eleitoral/blob/main/DistilGPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch accelerate tensorboard


In [ ]:
import pandas as pd

# Carregar o dataset
with open('codigo_eleitoral .txt', 'r', encoding='utf-8') as file:
    data = file.read()

# Dividir o texto em exemplos
def split_text(data, chunk_size=1024):
    chunks = [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]
    return chunks

examples = split_text(data)
df = pd.DataFrame(examples, columns=['text'])


In [ ]:
from datasets import Dataset

# Converter o dataframe para um dataset do Huggingface
dataset = Dataset.from_pandas(df)


In [ ]:
from transformers import GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
import torch.nn as nn

# Carregar o tokenizer e o modelo DistilGPT2
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
tokenizer.pad_token = tokenizer.eos_token

# Definindo a configuração personalizada
config = GPT2Config.from_pretrained('distilgpt2', attn_pdrop=0.2, resid_pdrop=0.2, embd_pdrop=0.2)

# Definindo o modelo personalizado com Dropout adicional
class CustomGPT2Model(GPT2LMHeadModel):
    def __init__(self, config):
        super(CustomGPT2Model, self).__init__(config)
        self.custom_dropout = nn.Dropout(p=0.3)

    def forward(self, input_ids=None, past_key_values=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, labels=None, use_cache=None, output_attentions=None, output_hidden_states=None, return_dict=None):
        # Chamando o forward do GPT2LMHeadModel
        outputs = super(CustomGPT2Model, self).forward(input_ids, past_key_values, attention_mask, token_type_ids, position_ids, head_mask, inputs_embeds, labels, use_cache, output_attentions, output_hidden_states, return_dict)

        # Aplicando Dropout personalizado aos logits
        logits = outputs.logits
        logits = self.custom_dropout(logits)

        return outputs

# Carregar o modelo com a configuração personalizada
model = CustomGPT2Model.from_pretrained('distilgpt2', config=config)


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=1024)

tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4)  # Utilizar múltiplos processos para acelerar

# Adicionar os rótulos ao dataset tokenizado
def add_labels(examples):
    examples["labels"] = examples["input_ids"].copy()
    return examples

tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)

# Dividir o dataset em treinamento e validação
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]


In [ ]:
import torch
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    # Remover o padding e calcular o F1-Score
    true_predictions = [p for p, l in zip(predictions.flatten(), labels.flatten()) if l != -100]
    true_labels = [l for l in labels.flatten() if l != -100]
    f1 = f1_score(true_labels, true_predictions, average="weighted")
    return {"f1": f1}

In [ ]:
from transformers import Trainer, TrainingArguments, IntervalStrategy

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=300,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=2e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    evaluation_strategy=IntervalStrategy.STEPS,
    eval_steps=500,
    fp16=True,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=False,
    report_to="tensorboard",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
# Caminho da pasta onde você quer salvar o modelo no Google Drive
output_dir = '/content/drive/My Drive/fine_tuned_distilgpt2'

# Criar o diretório se não existir
os.makedirs(output_dir, exist_ok=True)

# Salvar o modelo e o tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

In [ ]:
#Monitorar o Treinamento
%load_ext tensorboard
%tensorboard --logdir ./logs